In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import lightgbm
import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: cannot import name 'issparse' from 'scipy.sparse' (unknown location)

In [6]:
!conda install numpy scipy joblib scikit-learn --force-reinstall

'conda' is not recognized as an internal or external command,
operable program or batch file.


In [74]:
inp = "/kaggle/input/santander-customer-transaction-prediction"
train_data = pd.read_csv(inp + '/train.csv')


In [75]:
train_data

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001


In [76]:
X  = train_data.iloc[:,2:]
y = train_data.iloc[:,1]
y

0         0
1         0
2         0
3         0
4         0
         ..
199995    0
199996    0
199997    0
199998    0
199999    0
Name: target, Length: 200000, dtype: int64

In [77]:
from sklearn.preprocessing import StandardScaler

In [78]:
scale=StandardScaler()
X=scale.fit_transform(X)

In [79]:
from sklearn.preprocessing import LabelEncoder

In [80]:
sc = LabelEncoder()
y = sc.fit_transform(y)

In [81]:

X = pd.DataFrame(X)

In [84]:
y = pd.Series(y)

In [86]:
y.shape

(200000,)

In [94]:
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size=0.7,random_state=6)

In [95]:
y_train

96484     0
113038    0
74327     0
34142     0
98880     0
         ..
195949    0
4714      0
41187     0
117449    0
162698    0
Length: 60000, dtype: int64

In [96]:
train_data = lightgbm.Dataset(X_train,label=y_train)
valid_data = lightgbm.Dataset(X_test,label=y_test)

In [97]:
parameters = {'objective':'binary',
              'metric':'auc',
              'boosting':'dart',
              'num_leaves':72,
              'feature_fraction':0.5,
              "bagging_fraction": 0.5,
              'bagging_freq': 20,
              'learning_rate':0.1   
}

In [104]:
model = lightgbm.train(parameters,train_data,valid_sets=valid_data,num_boost_round=100)

[LightGBM] [Info] Number of positive: 6188, number of negative: 53812
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.094194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.103133 -> initscore=-2.162885
[LightGBM] [Info] Start training from score -2.162885
[1]	valid_0's auc: 0.64405
[2]	valid_0's auc: 0.679208
[3]	valid_0's auc: 0.700514
[4]	valid_0's auc: 0.709261
[5]	valid_0's auc: 0.715019
[6]	valid_0's auc: 0.726322
[7]	valid_0's auc: 0.732189
[8]	valid_0's auc: 0.735505
[9]	valid_0's auc: 0.744709
[10]	valid_0's auc: 0.74912
[11]	valid_0's auc: 0.754123
[12]	valid_0's auc: 0.756285
[13]	valid_0's auc: 0.761306
[14]	valid_0's auc: 0.767038
[15]	valid_0's auc: 0.769969
[16]	valid_0's auc: 0.771937
[17]	valid_0's auc: 0.775241
[18]	valid_0's auc: 

In [106]:
y_train_predict = model.predict(X_train)
y_valid_predict = model.predict(X_test)

In [112]:
y_train_predict


96484     0
113038    0
74327     0
34142     0
98880     0
         ..
195949    0
4714      0
41187     0
117449    0
162698    0
Length: 60000, dtype: int64

In [113]:
y_train

96484     0
113038    0
74327     0
34142     0
98880     0
         ..
195949    0
4714      0
41187     0
117449    0
162698    0
Length: 60000, dtype: int64

In [128]:
roc_auc_score(y_test,y_valid_predict)




0.5195164192682654

In [127]:
y_valid_predict

array([0., 0., 0., ..., 0., 0., 0.])

In [126]:
for i in range(len(y_valid_predict)):
    if y_valid_predict[i]>=0.5:
        y_valid_predict[i]=1
    else:
        y_valid_predict[i]=0
        

In [129]:
accuracy_score(y_test,y_valid_predict)

0.9040571428571429

In [130]:
f1_score(y_test,y_valid_predict)

0.07582220999036743